# Brief introduction to linear algebra using PGA
Ted Corcovilos, 2021-03-18

In [1]:
# import libraries
from sympy import *
from galgebra.ga import Ga
# setup notebook pretty printing
init_printing()

In [2]:
# set up the algebra
pga3coords = (w,x,y,z) = symbols('w x y z', real=True)
pga3 = Ga('e_0 e_1 e_2 e_3',g=[0,1,1,1], coords=pga3coords)

e0, e1, e2, e3 = pga3.mv()

!!!!If I**2 = 0, I cannot be normalized!!!!


In [3]:
# define some useful functions for PGA
def J3(x):
    # J map for pga3 multivectors to calculate the orthogonal complement
    coef_list = x.blade_coefs()
    # flip sign of e02 and e1
    signs = [1,
            1,-1,1,-1,
            1,-1,1,1,-1,1,
            1,-1,1,-1,
            1]
    size = len(signs)
    return pga3.mv(sum(signs[mm]*coef_list[mm]*flatten(pga3.blades)[size-1-mm] for mm in range(size)))

def vee3(a,b):
    # regressive product for pga3
    return J3(J3(a)^J3(b))

def norm(x):
    # norm.  Note that this does not keep the sign.
    return sqrt(((~x)*x).scalar())

def norm_ideal(x):
    # ideal norm.  Note that this does not keep the sign.
    return sqrt((~J3(x)*J3(x)).scalar())

def normalize(x):
    # normalize a MV
    return x/norm(x)

In [4]:
t= symbols('t', real=True) # real parameter to use later

## Systems of linear equations
Each linear equation may be represented by a vector in PGA.

Let's solve the system
$$
\begin{aligned}
a:&& x + \phantom{2}y + z -1 &= 0 \\
b:&& 2x + \phantom{2}y + z +0&= 0 \\
c:&& x -2y -z -2 &= 0
\end{aligned}
$$


In [5]:
a = e1 + e2 + e3 - e0
b = 2*e1 + e2 + e3
c = e1 -2*e2-e3 +2*e0 

The solution is the intersection point of the 3 planes described by each equation above.

In [6]:
a^b^c

3*e_0^e_1^e_2 + e_0^e_1^e_3 - e_0^e_2^e_3 - e_1^e_2^e_3

The trivector terms describe a point.  It's easier to read off if we normalize it and look at its orthogonal complement:

In [7]:
J3(a^b^c) 

e_0 - e_1 - e_2 + 3*e_3

The coefficient of $e_1$ is the $x$ value, etc.  So the solution of the original system of equations is
$$
\begin{aligned}
x&=-1\\
y&=-5\\
z&=3
\end{aligned}
$$

In [8]:
# Let's define p as a generic point
p=J3(x*e1 + y*e2 + z*e3 + e0)
p

-z*e_0^e_1^e_2 + y*e_0^e_1^e_3 - x*e_0^e_2^e_3 + e_1^e_2^e_3

We can represent this as equations for x,y,z by setting terms of the expression below equal to zero: $p\vee(a\wedge b\wedge c)=0$.  PGA is a Regressive Product Null Space (RPNS) representation.

In [9]:
vee3(p,a^b^c)

(-3*y - z)*e_0^e_1 + (3*x + z)*e_0^e_2 + (x - y)*e_0^e_3 + (z - 3)*e_1^e_2 + (-y - 1)*e_1^e_3 + (x + 1)*e_2^e_3

Setting the line above equal to zero gives a redundant set of equations, but looking at the line three terms we can read off
$$
\begin{aligned}
x&=-1\\
y&=-5\\
z&=7
\end{aligned}
$$

## Under-defined system
Let's try something different.  Let's do an under-defined system.
$$
\begin{aligned}
a:&& x + y + z -1 &= 0 \\
b:&& 2x + y + z +0&= 0 \\
%x -2y -z &= 2
\end{aligned}
$$

Again, the solution is $a\wedge b$:

In [10]:
a^b

-2*e_0^e_1 - e_0^e_2 - e_0^e_3 - e_1^e_2 - e_1^e_3

This is the Plücker representation of a line in 3D space.  As before, we can get this into a more familiar form by computing $p \vee (a\wedge b)=0$.

In [11]:
vee3(p,a^b)

(-2*x - y - z)*e_0 + (-y - z + 2)*e_1 + (x + 1)*e_2 + (x + 1)*e_3

### Parametric form of the line
(TODO generalize the method to higher dimensional solution spaces.)

The solution to the system above is a line of points: $x=-1, y=-z+2$.

To put this in parametric form, we need the direction of the line $d$ and any point on the line $P$:
$$
\begin{aligned}
d &= a\wedge b \wedge e_0 \\
P &= d^* \wedge (a\wedge b)
\end{aligned}
$$

Then the line consists of the points $P + dt$ for a scalar value $t$.

In [12]:
d=a^b^e0 # direction of the line
d

-e_0^e_1^e_2 - e_0^e_1^e_3

In [13]:
P = J3(d)^(a^b) # a point on the line
P

-2*e_0^e_1^e_2 + 2*e_0^e_1^e_3 + 2*e_0^e_2^e_3 + 2*e_1^e_2^e_3

In [14]:
(P+d*t)

(-t - 2)*e_0^e_1^e_2 + (2 - t)*e_0^e_1^e_3 + 2*e_0^e_2^e_3 + 2*e_1^e_2^e_3

Or, translating to coordinates and rescaling $t$:
$$
x = -1,\quad
y = 1-t, \quad z=1+t
$$

## Over-defined system
Let's look at an over-defined system in 2D:
$$
\begin{aligned}
x&= 0\\
y&= 0\\
x+y&= 1
\end{aligned}
$$
This clearly has no solution, but can we find a "best fit"?
One way to answer is to sum the 3 normalized intersection points.

(Q: what happens if I use the _unnormalized_ points? Is this useful?)

In [15]:
sum([e1^e2,-e2^(e1+e2-e0),-(e1+e2-e0)^e1])/3

e_0^e_1/3 - e_0^e_2/3 + e_1^e_2

To generalize to larger systems, for an _N_-dimensional space with _m_ equations, form all $\binom{m}{N}$ of the _N_-wise wedge products, normalize, and sum them.

This method also works when two or more of the (hyper-)planes are parallel.  The intersection point is then ideal, and that point can be discarded before summing.

(Q: does this give the same result as the least-squares solution, using pseudo-inverse?  It seems like that is different?)

## Gaussian elimination
One of the standard solution techniques in linear algebra is Gaussian elimination.  What does that look like in PGA?  It's almost trivial.

Let's go back to our earlier problem:
$$
\begin{aligned}
a:&&x + \phantom{2}y + z -1 &= 0 \\
b:&&2x + \phantom{2}y + z +0 &= 0 \\
c:&&x -2y -z -2 &= 0
\end{aligned}
$$

The idea of Gaussian elimination is to add a scalar multiple of one row to another row to zero out coefficients in the lower left triangle of the matrix.

For example, we could replace row $b$ with $b-2a$ and row $c$ with $c-a$, to eliminate the coefficients of $x$ in the bottom two rows:

In [16]:
b-2*a, c-a

$$
\begin{aligned}
a:&&x + \phantom{3}y +\phantom{2} z -1 &= 0 \\
b-2a:&&0x  -\phantom{3}y - \phantom{2}z +2&= 0 \\
c-\phantom{2}a:&&0x -3y -2z -1 &= 0
\end{aligned}
$$

Then add -3 times the second row to the third row:

In [17]:
-3*(b-2*a)+(c-a)

-3*e_0 + e_3

$$
\begin{aligned}
a:&&x + y + \phantom{0}z -1 &= 0 \\
b-2a:&&0x  -y - \phantom{0}z +2&= 0 \\
5a-3b+c:&&0x+ 0y +z -3 &= 0
\end{aligned}
$$

But the properties of the exterior product make this unnecessary.
$$
a\wedge(b-2a)\wedge(5a-3b+c) = a\wedge b \wedge c,
$$
so we are back where we started.

### Gaussian elimination as a series of rotations
We can also take a more geometric approach by thinking of Gaussian elimination as a series of rotations of the planes about their common lines of intersections.

In [18]:
test=normalize(1+normalize(a)*normalize(b))

In [19]:
X=e1; Y=e2; Z=e3

So, each step can take the form (for example)
$
b' = (e_1\cdot b)a
$.  How do I gradually turn this on?

In [20]:
# Step 1 is this rotor
test=normalize(1+normalize(b)*normalize(X|b*a))
# I need the log of this.  It's a rotation, so should be of the form
# cos(a/2)+sin(a/2)B

In [21]:
angle=acos(test[0].obj)
B = test[2]/sin(angle)
cos(angle)+sin(angle)*B

(sqrt(2) + sqrt(6))/(2*sqrt(sqrt(3) + 3)) + sqrt(2)*e_0^e_1/sqrt(sqrt(3) + 3) + sqrt(2)*e_0^e_2/(2*sqrt(sqrt(3) + 3)) + sqrt(2)*e_0^e_3/(2*sqrt(sqrt(3) + 3)) + sqrt(2)*e_1^e_2/(2*sqrt(sqrt(3) + 3)) + sqrt(2)*e_1^e_3/(2*sqrt(sqrt(3) + 3))

In [22]:
#Step 2
c-a, X|c*a

In [23]:
#Step 3
Y|(X|c)

0

### TODOs
* add figures, highlight geometry
* describe rotation picture of Gaussian elimination
* projections, rejections, reflections
* emphasize linear combinations of planes keep their intersection invariant
* add norm and normalize functions
* use macros to make the text automatically update if the equations are changed